In [ ]:
!pip install google-cloud-bigquery

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime, timedelta
import pandas as pd

date_format = '%Y-%m-%d'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
credentials_file = '/content/drive/MyDrive/Colab Notebooks/share/datascience-922910cdee23.json'
project_id = 'datascience-248215'
credentials = service_account.Credentials.from_service_account_file(credentials_file)

bq_client = bigquery.Client(credentials=credentials, project=project_id)

In [ ]:
brands = {
    1: 'MUY',
    2: 'El Original',
    4: 'Pixi',
    5: 'Just Burgers',
    6: 'Tremendo',
    7: 'The Cut',
    8: 'Pecado Natural',
    9: 'Robinfood',
    10: 'Taco Tlan',
    11: 'Tributo'
}
brands

{1: 'MUY',
 2: 'El Original',
 4: 'Pixi',
 5: 'Just Burgers',
 6: 'Tremendo',
 7: 'The Cut',
 8: 'Pecado Natural',
 9: 'Robinfood',
 10: 'Taco Tlan',
 11: 'Tributo'}

In [ ]:
date_to_process = datetime.strptime('2021-08-01', '%Y-%m-%d')
country_id = 1
lower_date_range = date_to_process - timedelta(days=7)
query = ''' 
SELECT
  *
FROM
  `datascience-248215.muy.ds_orders`
WHERE
  company_id = {0}
  AND fecha <= '{1}'
  AND fecha > '{2}' '''
sales_data = bq_client.query(query.format(country_id, date_to_process.strftime(date_format), lower_date_range.strftime(date_format))).to_dataframe()
sales_data.head(5)

,company_id,marca_id,medio,canal,store_id,store_name,fecha,orders
0,1,1,OFF-PREMISE,IFOOD,2,MUY 98,2021-07-26,1
1,1,1,OFF-PREMISE,IFOOD,3,MUY 79,2021-07-26,11
2,1,2,OFF-PREMISE,IFOOD,3,MUY 79,2021-07-26,9
3,1,6,OFF-PREMISE,IFOOD,3,MUY 79,2021-07-26,9
4,1,11,OFF-PREMISE,IFOOD,3,MUY 79,2021-07-26,5


In [ ]:
lower_date_range

datetime.datetime(2021, 7, 25, 0, 0)

In [ ]:
sales_data['weekday'] = sales_data.fecha.apply(lambda date: date.weekday())
#sales_data.head()

In [ ]:
brand_names = sales_data.marca_id.apply(lambda id: brands[id])
sales_data['brand_name'] = brand_names
#sales_data.head()

In [ ]:
sales_data.loc[sales_data.medio != 'OFF-PREMISE', 'canal'] = 'ON-PREMISE'

In [ ]:
platform_list = sales_data.canal.unique()
brand_list = sales_data.marca_id.unique()
store_list = sales_data.store_id.unique()
print(platform_list)
print(brand_list)
print(store_list)

['IFOOD' 'ON-PREMISE' 'RAPPI' 'JUSTO']
[ 1  2  6 11  4  8]
[ 2  3  4  5  6  8  9 10 11 12 13 15 18 19 20 21 22 25 26 27 29 30 33 39
 40 41 42 44 45 46 50 51 52 58 59 60 61 62 63 67 69 70 71 74 75 76 77 78
 84 85 88]


In [ ]:
sales_by_brand = sales_data.groupby(['marca_id']).orders.sum()
sales_by_brand

marca_id
1     27670
2       950
4      3192
6      1627
8      1436
11     2165
Name: orders, dtype: int64

In [ ]:
sales_by_platform = sales_data.groupby(['marca_id', 'canal']).orders.sum().reset_index()
sales_by_platform = sales_by_platform.rename(columns={'orders': 'share', 'canal': 'platform'})
for brand in brand_list:
  share = sales_by_platform.loc[sales_by_platform.marca_id == brand].share / sales_by_brand.loc[brand]
  sales_by_platform.loc[sales_by_platform.marca_id == brand, 'share'] = share
  
share_by_platform = sales_by_platform.set_index(['marca_id', 'platform'])
#sales_by_platform

In [ ]:
sales_by_platform = sales_data.groupby(['marca_id', 'canal']).orders.sum()
sales_by_platform

marca_id  canal     
1         IFOOD          5250
          JUSTO            56
          ON-PREMISE    17631
          RAPPI          4733
2         IFOOD           741
          ON-PREMISE       71
          RAPPI           138
4         IFOOD          2509
          JUSTO             2
          ON-PREMISE      432
          RAPPI           249
6         IFOOD          1060
          JUSTO             2
          ON-PREMISE      414
          RAPPI           151
8         IFOOD           873
          JUSTO             9
          ON-PREMISE      329
          RAPPI           225
11        IFOOD          1096
          JUSTO            15
          ON-PREMISE      797
          RAPPI           257
Name: orders, dtype: int64

In [ ]:
sales_by_platform = sales_data.groupby(['marca_id', 'canal']).orders.sum()
sales_by_store = sales_data.groupby(['marca_id', 'canal', 'store_id']).orders.sum().reset_index()
sales_by_store = sales_by_store.rename(columns={'orders': 'share', 'canal': 'platform'})
for brand in brand_list:
  brand_slice = sales_by_store.marca_id == brand
  for platform in platform_list:
    platform_slice = brand_slice & (sales_by_store.platform == platform)
    if sum(platform_slice) == 0:
      continue
    share = sales_by_store.loc[platform_slice].share / sales_by_platform.loc[(brand, platform)]
    sales_by_store.loc[platform_slice, 'share'] = share
#shares_by_store = sales_by_store.set_index(['marca_id', 'platform', 'store_id'])
#shares_by_store

shares_by_store=sales_by_store
shares_by_store

,marca_id,platform,store_id,share
0,1,IFOOD,2,0.000190
1,1,IFOOD,3,0.016952
2,1,IFOOD,5,0.024190
3,1,IFOOD,6,0.020000
4,1,IFOOD,8,0.015619
...,...,...,...,...
517,11,RAPPI,69,0.050584
518,11,RAPPI,70,0.050584
519,11,RAPPI,71,0.089494
520,11,RAPPI,76,0.027237


In [ ]:
shares_by_store[(shares_by_store.store_id == 39) & (shares_by_store.marca_id == 1 )]

,marca_id,platform,store_id,share
22,1,IFOOD,39,0.070286
53,1,JUSTO,39,0.035714
80,1,ON-PREMISE,39,0.015371
125,1,RAPPI,39,0.052821


In [ ]:
sales_by_platform = sales_data.groupby(['marca_id', 'canal']).orders.sum().reset_index()
sales_by_store = sales_data.groupby(['marca_id', 'canal', 'store_id']).orders.sum().reset_index()
sales_by_store = sales_by_store.rename(columns={'orders': 'share', 'canal': 'platform'})
for row in sales_by_platform.itertuples():
  for row1 in sales_by_store.itertuples():
    if((row.marca_id==row1.marca_id) & (row.canal==row1.platform)):
      share=row1.share/row.orders
      sales_by_store.loc[row1.Index, "share"] =share
#shares_by_store1 = sales_by_store.set_index(['marca_id', 'platform', 'store_id'])
shares_by_store1=sales_by_store
#shares_by_store1


In [ ]:
shares_by_store1[(shares_by_store1.store_id == 39) & (shares_by_store1.marca_id == 1 )]

,marca_id,platform,store_id,share
22,1,IFOOD,39,0.070286
53,1,JUSTO,39,0.035714
80,1,ON-PREMISE,39,0.015371
125,1,RAPPI,39,0.052821


In [ ]:
sales_by_store = sales_data.groupby(['marca_id', 'canal', 'store_id']).orders.sum()

sales_by_weekday = sales_data.groupby(['marca_id', 'canal', 'store_id', 'weekday']).orders.sum().reset_index()
sales_by_weekday = sales_by_weekday.rename(columns={'orders': 'share', 'canal': 'platform'})
for brand in brand_list:
  brand_slice = sales_by_weekday.marca_id == brand
  for platform in platform_list:
    platform_slice = brand_slice & (sales_by_weekday.platform == platform)
    for store in store_list:
      store_slice = platform_slice & (sales_by_weekday.store_id == store)
    if sum(store_slice) == 0:
      continue
    share = sales_by_weekday.loc[store_slice].share / sales_by_store.loc[(brand, platform, store)]
    sales_by_weekday.loc[store_slice, 'share'] = share


shares_by_weekday = sales_by_weekday.copy()
shares_by_weekday['level'] = 4

In [ ]:
sales_by_store[(sales_by_store.store_id == 39) & (sales_by_store.marca_id == 1 )]

AttributeError: ignored

In [ ]:
sales_by_weekday = sales_data.groupby(['marca_id', 'canal', 'store_id', 'weekday']).orders.sum().reset_index()
sales_by_store = sales_data.groupby(['marca_id', 'canal', 'store_id']).orders.sum().reset_index()
sales_by_weekday["share"]=0
for row in sales_by_store.itertuples():
  for row1 in sales_by_weekday.itertuples():
    if((row.marca_id==row1.marca_id) & (row.canal==row1.canal)& (row.store_id==row1.store_id) ):
      share=row1.orders/row.orders
      sales_by_weekday.loc[row1.Index, "share"] =share
sales_by_weekday

,marca_id,canal,store_id,weekday,orders,share
0,1,IFOOD,2,0,1,1.000000
1,1,IFOOD,3,0,11,0.123596
2,1,IFOOD,3,1,9,0.101124
3,1,IFOOD,3,2,21,0.235955
4,1,IFOOD,3,3,23,0.258427
...,...,...,...,...,...,...
2786,11,RAPPI,85,0,1,0.090909
2787,11,RAPPI,85,2,2,0.181818
2788,11,RAPPI,85,3,3,0.272727
2789,11,RAPPI,85,4,3,0.272727


In [ ]:
sales_by_weekday[(sales_by_weekday.store_id == 2) & (sales_by_weekday.marca_id == 1 )]

,marca_id,canal,store_id,weekday,orders,share
0,1,IFOOD,2,0,1,1.0


In [ ]:
sales_by_weekday=sales_by_weekday.reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2791 entries, 0 to 2790
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     2791 non-null   int64  
 1   marca_id  2791 non-null   int64  
 2   canal     2791 non-null   object 
 3   store_id  2791 non-null   int64  
 4   weekday   2791 non-null   int64  
 5   orders    2791 non-null   int64  
 6   share     2791 non-null   float64
dtypes: float64(1), int64(5), object(1)
memory usage: 152.8+ KB


# Calculo de share

In [ ]:
date_format = '%Y-%m-%d'
country_id = 1

credentials_file = '/content/drive/MyDrive/Colab Notebooks/share/datascience-922910cdee23.json'
project_id = 'datascience-248215'
credentials = service_account.Credentials.from_service_account_file(credentials_file)

bq_client = bigquery.Client(credentials=credentials, project=project_id)

brands = {
    1: 'MUY',
    2: 'El Original',
    4: 'Pixi',
    5: 'Just Burgers',
    6: 'Tremendo',
    7: 'The Cut',
    8: 'Pecado Natural',
    9: 'Robinfood',
    10: 'Taco Tlan',
    11: 'Tributo'
}

all_shares = pd.DataFrame()

date_to_process = datetime.strptime('2021-07-25', '%Y-%m-%d')
lower_date_range = date_to_process - timedelta(days=28)
query = ''' 
SELECT
  *
FROM
  `datascience-248215.muy.ds_orders`
WHERE
  company_id = {0}
  AND fecha <= '{1}'
  AND fecha > '{2}' '''
sales_data = bq_client.query(query.format(country_id, date_to_process.strftime(date_format), lower_date_range.strftime(date_format))).to_dataframe()
sales_data['weekday'] = sales_data.fecha.apply(lambda date: date.weekday())

brand_names = sales_data.marca_id.apply(lambda id: brands[id])
sales_data['brand_name'] = brand_names

sales_data.loc[sales_data.medio != 'OFF-PREMISE', 'canal'] = 'ON-PREMISE'

platform_list = sales_data.canal.unique()
brand_list = sales_data.marca_id.unique()
store_list = sales_data.store_id.unique()

sales_by_brand = sales_data.groupby(['marca_id']).orders.sum()
sales_by_platform = sales_data.groupby(['marca_id', 'canal']).orders.sum().reset_index()
sales_by_platform = sales_by_platform.rename(columns={'orders': 'share', 'canal': 'platform'})
for brand in brand_list:
  share = sales_by_platform.loc[sales_by_platform.marca_id == brand].share / sales_by_brand.loc[brand]
  sales_by_platform.loc[sales_by_platform.marca_id == brand, 'share'] = share
  
shares_by_platform = sales_by_platform.copy()
shares_by_platform['level'] = 2

all_shares = pd.concat([all_shares, shares_by_platform])

sales_by_platform = sales_data.groupby(['marca_id', 'canal']).orders.sum()

sales_by_store = sales_data.groupby(['marca_id', 'canal', 'store_id']).orders.sum().reset_index()
sales_by_store = sales_by_store.rename(columns={'orders': 'share', 'canal': 'platform'})
for brand in brand_list:
  brand_slice = sales_by_store.marca_id == brand
  for platform in platform_list:
    platform_slice = brand_slice & (sales_by_store.platform == platform)
    if sum(platform_slice) == 0:
      continue
    share = sales_by_store.loc[platform_slice].share / sales_by_platform.loc[(brand, platform)]
    sales_by_store.loc[platform_slice, 'share'] = share

shares_by_store = sales_by_store.copy()
shares_by_store['level'] = 3
all_shares = pd.concat([all_shares, shares_by_store])
sales_by_store = sales_data.groupby(['marca_id', 'canal', 'store_id']).orders.sum().reset_index()

sales_by_weekday = sales_data.groupby(['marca_id', 'canal', 'store_id', 'weekday']).orders.sum().reset_index()
sales_by_weekday = sales_by_weekday.rename(columns={'orders': 'share','canal': 'platform'})
for row in sales_by_store.itertuples():
  for row1 in sales_by_weekday.itertuples():
    if((row.marca_id==row1.marca_id) & (row.canal==row1.platform)& (row.store_id==row1.store_id) ):
      share=row1.share/row.orders
      sales_by_weekday.loc[row1.Index, "share"] =share

shares_by_weekday = sales_by_weekday.copy()
shares_by_weekday['level'] = 4

all_shares = pd.concat([all_shares, shares_by_weekday])
all_shares['country'] = 1

all_shares = all_shares.iloc[:, [3, 6, 0, 1, 4, 5, 2]]
all_shares.to_csv('shares.csv', index=False)



In [ ]:
lower_date_range

datetime.datetime(2021, 6, 27, 0, 0)

In [ ]:
import numpy as np
levels = 4
country_id = 1
marca_id = 1
orders = 27023
loaded_shares = pd.read_csv('shares.csv')
loaded_shares = loaded_shares[(loaded_shares.country == country_id) & (loaded_shares.marca_id == marca_id)]
level_2 = loaded_shares[loaded_shares.level == 2][['country', 'marca_id', 'platform', 'share']]
all_platforms = level_2.platform.unique()
level_2['orders'] = np.ceil(level_2.share * orders)

level_3 = loaded_shares[loaded_shares.level == 3][['country', 'marca_id', 'platform', 'store_id', 'share']]
all_stores = level_3.store_id.unique()

for platform in all_platforms:
  orders = level_2[level_2.platform == platform].orders.values[0]
  print(orders)
  platform_slice = level_3.platform == platform
  level_3.loc[platform_slice, 'orders'] = np.ceil(level_3[platform_slice].share * orders)

level_4 = loaded_shares[loaded_shares.level == 4][['country', 'marca_id', 'platform', 'store_id', 'weekday', 'share']]
for platform in all_platforms:
  platform_slice = level_4.platform == platform
  for store in all_stores:
    store_slice = platform_slice & (level_4.store_id == store)
    if sum(store_slice) == 0:
      continue
    orders = level_3[(level_3.platform == platform) & (level_3.store_id == store)].orders.values[0]
    level_4.loc[store_slice, 'orders'] = np.ceil(level_4[store_slice].share * orders)


5953.0
84.0
16586.0
4402.0


# Validar datos de calculo de share con valores reales 

In [ ]:
date_format = '%Y-%m-%d'
country_id = 1

credentials_file = '/content/drive/MyDrive/Colab Notebooks/share/datascience-922910cdee23.json'
project_id = 'datascience-248215'
credentials = service_account.Credentials.from_service_account_file(credentials_file)

bq_client = bigquery.Client(credentials=credentials, project=project_id)
brands = {
    1: 'MUY',
    2: 'El Original',
    4: 'Pixi',
    5: 'Just Burgers',
    6: 'Tremendo',
    7: 'The Cut',
    8: 'Pecado Natural',
    9: 'Robinfood',
    10: 'Taco Tlan',
    11: 'Tributo'
}
date_to_process = datetime.strptime('2021-08-01', '%Y-%m-%d')
lower_date_range = date_to_process - timedelta(days=7)
query = ''' 
SELECT
  *
FROM
  `datascience-248215.muy.ds_orders`
WHERE
  company_id = {0}
  AND fecha <= '{1}'
  AND fecha > '{2}' '''
sales_data = bq_client.query(query.format(country_id, date_to_process.strftime(date_format), lower_date_range.strftime(date_format))).to_dataframe()
sales_data['weekday'] = sales_data.fecha.apply(lambda date: date.weekday())
brand_names = sales_data.marca_id.apply(lambda id: brands[id])
sales_data['brand_name'] = brand_names
sales_data.loc[sales_data.medio != 'OFF-PREMISE', 'canal'] = 'ON-PREMISE'
platform_list = sales_data.canal.unique()
brand_list = sales_data.marca_id.unique()
store_list = sales_data.store_id.unique()
sales_by_brand = sales_data.groupby(['marca_id']).orders.sum()
sales_by_platform = sales_data.groupby(['marca_id', 'canal']).orders.sum().reset_index()
sales_by_store = sales_data.groupby(['marca_id', 'canal', 'store_id']).orders.sum().reset_index()
sales_by_weekday = sales_data.groupby(['marca_id', 'canal', 'store_id', 'weekday']).orders.sum().reset_index()


In [ ]:
lower_date_range

datetime.datetime(2021, 7, 25, 0, 0)

In [ ]:
sales_by_platform

,marca_id,canal,orders
0,1,IFOOD,5250
1,1,JUSTO,56
2,1,ON-PREMISE,17631
3,1,RAPPI,4733
4,2,IFOOD,741
5,2,ON-PREMISE,71
6,2,RAPPI,138
7,4,IFOOD,2509
8,4,JUSTO,2
9,4,ON-PREMISE,432


**Por canal-marca**

In [ ]:
level_2

,country,marca_id,platform,share,orders
0,1,1,IFOOD,0.220264,5953.0
1,1,1,JUSTO,0.003089,84.0
2,1,1,ON-PREMISE,0.613768,16586.0
3,1,1,RAPPI,0.162879,4402.0


In [ ]:
level_2.orders.sum()

27025.0

In [ ]:
canal_no_index =sales_by_platform.reset_index()
real_platform= canal_no_index[ (canal_no_index.marca_id == 1)]
real_platform

,index,marca_id,canal,orders
0,0,1,IFOOD,5250
1,1,1,JUSTO,56
2,2,1,ON-PREMISE,17631
3,3,1,RAPPI,4733


In [ ]:
real_platform.orders.sum()

27670

In [ ]:
level_2.orders.values / real_platform.orders.values

array([1.13390476, 1.5       , 0.9407294 , 0.9300655 ])

In [ ]:
 real_platform.orders.values/ level_2.orders.values 

array([0.88190828, 0.66666667, 1.06300494, 1.07519309])

**Por store-marca-canal**

In [ ]:
store_no_index = sales_by_store.reset_index()
real_store = store_no_index[(store_no_index.store_id == 39) & (store_no_index.marca_id == 1)]
real_store

,index,marca_id,canal,store_id,orders
22,22,1,IFOOD,39,369
53,53,1,JUSTO,39,2
80,80,1,ON-PREMISE,39,271
125,125,1,RAPPI,39,250


In [ ]:
predicted_store = level_3[level_3.store_id == 39]
predicted_store

,country,marca_id,platform,store_id,share,orders
48,1,1,IFOOD,39.0,0.069480,414.0
78,1,1,JUSTO,39.0,0.058462,5.0
105,1,1,ON-PREMISE,39.0,0.015936,265.0
150,1,1,RAPPI,39.0,0.050481,223.0


In [ ]:
a=real_store.orders.sum()
b=predicted_store.orders.sum()
if(a<b):
  print(real_store.orders.sum() / predicted_store.orders.sum())
else:
  print(predicted_store.orders.sum()/real_store.orders.sum() )

0.9834619625137817


In [ ]:
predicted_store.orders.values / real_store.orders.values

array([1.12195122, 2.5       , 0.97785978, 0.892     ])

In [ ]:
real_store.orders.values/predicted_store.orders.values 

array([0.89130435, 0.4       , 1.02264151, 1.12107623])

**Por store-marca-canal-dia**

In [ ]:
storeday = sales_by_weekday.reset_index()
real_storeday = storeday[(storeday.store_id == 39) & (storeday.marca_id == 1)]
real_storeday

,index,marca_id,canal,store_id,weekday,orders
141,141,1,IFOOD,39,0,47
142,142,1,IFOOD,39,1,53
143,143,1,IFOOD,39,2,65
144,144,1,IFOOD,39,3,73
145,145,1,IFOOD,39,4,42
146,146,1,IFOOD,39,5,48
147,147,1,IFOOD,39,6,41
323,323,1,JUSTO,39,0,1
324,324,1,JUSTO,39,5,1
486,486,1,ON-PREMISE,39,0,37


In [ ]:
predicted_storeday = level_4[level_4.store_id == 39]
predicted_storeday

,country,marca_id,platform,store_id,weekday,share,orders
720,1,1,IFOOD,39.0,0.0,0.154037,64.0
721,1,1,IFOOD,39.0,1.0,0.130435,54.0
722,1,1,IFOOD,39.0,2.0,0.095031,40.0
723,1,1,IFOOD,39.0,3.0,0.132919,56.0
724,1,1,IFOOD,39.0,4.0,0.145963,61.0
725,1,1,IFOOD,39.0,5.0,0.167081,70.0
726,1,1,IFOOD,39.0,6.0,0.174534,73.0
922,1,1,JUSTO,39.0,0.0,0.157895,1.0
923,1,1,JUSTO,39.0,1.0,0.210526,2.0
924,1,1,JUSTO,39.0,3.0,0.210526,2.0


In [ ]:
 predicted_storeday[predicted_storeday.platform == "ON-PREMISE"].orders.values

array([38., 31., 31., 35., 38., 43., 53.])

In [ ]:
 real_storeday[real_storeday.canal == "ON-PREMISE"].orders.values

array([37, 18, 29, 39, 26, 59, 63])

In [ ]:
 real_storeday[real_storeday.canal == "ON-PREMISE"].orders.values /  predicted_storeday[predicted_storeday.platform == "ON-PREMISE"].orders.values

array([0.97368421, 0.58064516, 0.93548387, 1.11428571, 0.68421053,
       1.37209302, 1.18867925])

In [ ]:
 predicted_storeday[predicted_storeday.platform == "ON-PREMISE"].orders.values /  real_storeday[real_storeday.canal == "ON-PREMISE"].orders.values 

array([1.02702703, 1.72222222, 1.06896552, 0.8974359 , 1.46153846,
       0.72881356, 0.84126984])